使用許諾条件

①	設計一次エネルギー消費量算定法に基づく計算方法をPythonによりコーディングし、Jupyter notebook形式で保存したファイル（以下、本ファイルと呼ぶ）の著作権は、一般社団法人日本サステナブル建築協会に帰属します。
Copyright©2019. 一般社団法人日本サステナブル建築協会
All rights reserved. 

②	本ファイルは、住戸の設計一次エネルギー消費量算定方法に係る「平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）2.1算定方法（第四章第七節 温水暖房）」に示す計算方法に基づき、国土交通省国土技術政策総合研究所・国立研究開発法人建築研究所、株式会社ユーワークス、株式会社住環境計画研究所が作成したものです。

③	本ファイルは、計算方法に示す計算過程の確認及び調査研究等に活用することを目的として作成されたものであり、一般社団法人日本サステナブル建築協会は著作権について放棄はしませんが、使用する各人の自己責任において仕様を変更することを妨げるものではありません。ただし、本ファイルによる計算結果を住宅の省エネルギー基準で規定する設計一次エネルギー消費量の結果として公的な申請に用いることはできません。

④	本ファイルを変更するかしないかを問わず、以下の条件を満たす場合に限り、頒布および使用が許可されます。
1.	頒布物の付属ドキュメント等の資料に、上記の著作権表示、本条件一覧、および下記免責条項を含めること。
2.	書面による特別の許可なしに、本ファイルから派生した製品の宣伝または販売の促進に、一般社団法人日本サステナブル建築協会、国土技術政策総合研究所・国立研究開発法人建築研究所および株式会社ユーワークス、株式会社住環境計画研究所の名前またはコントリビューターの名前を使用してはならない。

⑤	本ファイルは、著作権者およびコントリビューターによって「現状のまま」提供されており、明示黙示を問わず、商業的な使用可能性、および特定の目的に対する適合性に関する暗黙の保証も含め、またそれに限定されない、いかなる保証もありません。著作権もコントリビューターも、事由のいかんを問わず、損害発生の原因いかんを問わず、かつ責任の根拠が契約であるか厳格責任であるか（過失その他の）不法行為であるかを問わず、仮にそのような損害が発生する可能性を知らされていたとしても、本ファイルの使用によって発生した（代用品または代用サービスの調達、使用の喪失、データの喪失、利益の喪失、業務の中断も含め、またそれに限定されない）直接損害、間接損害、偶発的な損害、特別損害、懲罰的損害、または結果損害について、一切の責任を負わないものとします。



In [ ]:
import nbimporter
import numpy as np
import s04_07_m_max_capacity_of_radiator as appendix_M

## 付録K　ファンコンベクター  / Appendix K Fan Convector

### K.1 記号及び単位 / Symbol and unit

省略

ommision

### K.2 消費電力量 / Power consumption

$Q_{T,rad} \leq Q_{min,FC} $ の場合

In the case of $Q_{T,rad} \leq Q_{min,FC} $

$$\large
E_{E,rad} = P_{min,FC} \times \frac{Q_{T,H,rad}}{Q_{min,H,FC}} \times 10^{-3}
$$

$Q_{min,FC} < Q_{T,rad} \leq Q_{max,H,FC} $ の場合

In the case of $Q_{min,FC} < Q_{T,rad} \leq Q_{max,H,FC} $ 

$$
E_{E,rad} = \bigg ( P_{min,FC} \times \frac{Q_{max,H,FC}-Q_{T,H,rad}}{Q_{max,H,FC}-Q_{min,H,FC}} + P_{max,FC} \times  \frac{Q_{T,H,rad}-Q_{min,H,FC}}{Q_{max,H,FC}-Q_{min,H,FC}} \bigg) \times 10^{-3}
$$

$Q_{max,H,FC} \leq Q_{T,rad}$ の場合

In the case of $Q_{max,H,FC} \leq Q_{T,rad}$ 

$$\large
E_{E,rad} = P_{max,FC} \times 10^{-3}
$$

$E_{E,rad}$：1 時間当たりの放熱器の消費電力量(kWh/h) / Power consumption of radiator per hour  
$Q_{T,H,rad}$：1 時間当たりの放熱器の処理暖房負荷(MJ/h) / Radiator processing heating load per hour  
$Q_{max,H,FC}$：1 時間当たりのファンコンベクターの最大暖房出力(MJ/h) / Maximum heating power per fan convector per hour  
$Q_{min,H,FC}$：1 時間当たりのファンコンベクターの最小出力(MJ/h) / Minimum output of fan convector per hour  
$P_{max,FC}$：ファンコンベクターの最大消費電力(W) / Maximum power consumption of fan convector  
$P_{min,FC}$：ファンコンベクターの最小消費電力(W) / Minimum power consumption of fan convector     

In [ ]:
# 放熱器の消費電力量 (1)
def get_E_E_rad(region, mode, A_HCZ, R_type, Theta_SW, Q_T_H_rad):
    # ファンコンベクターの最大能力及び最小能力
    q_max_FC = get_q_max_FC(region, mode, A_HCZ, R_type)
    q_min_FC = get_q_min_FC(q_max_FC)

    # ファンコンベクターの最大暖房出力及び最小暖房出力
    Q_max_H_FC = get_Q_max_H_FC(Theta_SW, q_max_FC)
    Q_min_H_FC = get_Q_min_H_FC(Theta_SW, q_min_FC)

    # ファンコンベクターの最大消費電力及び最小消費電力
    P_max_FC = get_P_max_FC(q_max_FC)
    P_min_FC = get_P_min_FC(q_min_FC)

    # (1a)
    tmp_1a = P_min_FC * (Q_T_H_rad / Q_min_H_FC) * 10 ** (-3)
    tmp_1a[np.logical_not(Q_T_H_rad <= Q_min_H_FC)] = 0

    # (1b)
    tmp_1b = (P_min_FC * (Q_max_H_FC - Q_T_H_rad) / (Q_max_H_FC - Q_min_H_FC) + P_max_FC * (Q_T_H_rad - Q_min_H_FC) / (
                Q_max_H_FC - Q_min_H_FC)) * 10 ** (-3)
    tmp_1b[np.logical_not(np.logical_and(Q_min_H_FC < Q_T_H_rad, Q_T_H_rad < Q_max_H_FC))] = 0

    # (1c)
    tmp_1c = P_max_FC * 10 ** (-3) * np.ones_like(Q_T_H_rad)
    tmp_1c[np.logical_not(Q_max_H_FC <= Q_T_H_rad)] = 0

    E_E_rad = tmp_1a + tmp_1b + tmp_1c
    return E_E_rad


### K.3 温水供給運転率 / Hot water supply operation rate

$$\large
r_{WS,rad} = \frac{Q_{T,H,rad}}{Q_{min,H,FC}}
$$

$r_{WS,rad}$：1 時間平均の放熱器の温水供給運転率 / 1 hour average radiator hot water supply operation rate  

In [ ]:
# 最大暖房出力
def get_Q_max_H_rad(Theta_SW, q_max_FC):
    return get_Q_max_H_FC(Theta_SW, q_max_FC)

### K.4 最大暖房出力 / Maximum heating output

省略

ommision

### K.5 ファンコンベクターの最大暖房出力及び最小暖房出力 / Fan convector maximum heating output and minimum heating output

$$\large
Q_{max,H,FC} = q_{max,FC} \times \frac{\theta_{SW} - 20}{60 - 20} \times 3600 \times 10^{-6}
$$

$$\large
Q_{min,H,FC} = q_{min,FC} \times \frac{\theta_{SW} - 20}{60 - 20} \times 3600 \times 10^{-6}
$$

$q_{max,FC}$：ファンコンベクターの最大能力(W) / Maximum ability of fan convector   
$q_{min,FC}$：ファンコンベクターの最小能力(W) / Minimum ability of fan convector  
$\theta_{SW}$：往き送水温度(℃) / Forward water temperature  

In [ ]:
# ファンコンベクターの最大暖房出力 (3a)
def get_Q_max_H_FC(Theta_SW, q_max_FC):
    return q_max_FC * (Theta_SW - 20) / (60 - 20) * 3600 * 10 ** (-6)


# ファンコンベクターの最小暖房出力 (3b)
def get_Q_min_H_FC(Theta_SW, q_min_FC):
    return q_min_FC * (Theta_SW - 20) / (60 - 20) * 3600 * 10 ** (-6)

$$\large
q_{min,FC} = 0.4859 \times q_{max,FC}
$$

In [ ]:
# ファンコンベクターの最大能力
def get_q_max_FC(region, mode, A_HCZ, R_type):
    # 付録Mに定める放熱器の最大能力 q_max_rad に等しいものとする
    return appendix_M.get_q_max_rad(region, mode, A_HCZ, R_type)

### K.6 ファンコンベクターの最大消費電力及び最小消費電力 / Maximum power consumption and minimum power consumption of fan convector

$$\large
P_{max,FC} = 7.564 \times 10^{-3} \times q_{max,FC}
$$

$$\large
P_{min,FC} = 7.783 \times 10^{-3} \times q_{min,FC}
$$

In [ ]:
# ファンコンベクターの最大能力
def get_q_max_FC(region, mode, A_HCZ, R_type):
    # 付録Mに定める放熱器の最大能力 q_max_rad に等しいものとする
    return appendix_M.get_q_max_rad(region, mode, A_HCZ, R_type)


# ファンコンベクターの最小能力 (4)
def get_q_min_FC(q_max_FC):
    return 0.4859 * q_max_FC